In [22]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random

In [23]:
def createList(r1, r2):
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [24]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1):
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else:
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [25]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [26]:
class Predator():
    
    def __init__(self, location):
        self.pos = location
        #making sure that predator doesn't start where the agent is
        
    def pred_move(self, g, agentpos):
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        self.pos = shortest_path[new_path][0]
        return self

    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60:
            self.pos = shortest_path[new_path][0]
        else:
            self.pos = random.choice(neighbors)
        return self

In [27]:
class Prey():
    
    def __init__(self, location):
        self.pos = location
        #making sure prey doesn't start where agent is
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [7]:
def Agent1(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    
    while (fail == 0 and steps <= 100):
        
        #print("Positions: " + str(agent_pos) + ", " + str(predator.pos) + ", "+ str(prey.pos))
        # getting all possible next positions of agent 1
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and the neighbor they would move to for that path
        #keeping separate moves lists for each of dr cowans conditions
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = [] #initializing path from agents current position to prey
        path_to_predator = [] #initializing path from agents current position to predator
        curr_to_pred = len(BFS(agent_pos, predator.pos, g.graph)) #lenth of path from agents current position to predator
        curr_to_prey = len(BFS(agent_pos, prey.pos, g.graph)) #path from agents current position to prey
        
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, prey.pos, g.graph)
            path_to_predator = BFS(neighbor, predator.pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred): #agent 1 condition 1
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred): #agent 1 condition 2
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred): #agent 1 condition 3
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred): #agent 1 condition 4
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred): #agent 1 condition 5
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred): #agent 1 condition 6
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0): #if there are moves that satisfy condition 1
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1])) #sorting by distance to prey
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0): #if no moves satisfying condition 1, are there are moves that satisfy condition 2?
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1])) #sorting by distance to prey
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        #so on and so forth from here on according to dr cowans condition

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        #if none of the lists have a move available, means the agent should sit still and pray
        #if none of the if else cases were executed, the same agent  position will get appended again
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move predator and prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator.pred_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        
    #fail if timeout error    
    if(steps > 5000):
        fail = 2     
    return fail, path

In [8]:
def Agent2(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location

    while(fail==0 and steps <= 100):
        neighbours = g.graph[agent_pos] #getting neighbours
        amazing_moves = [] #keeping list of moves that only satisfy Agent 1s first condition
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predator.pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, prey.pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)] #keeping list of all possible moves available to agent at any time including staying in place

        
        for neighbour in neighbours:
            path_to_prey = BFS(neighbour, prey.pos, g.graph)
            path_to_pred = BFS(neighbour, predator.pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2) 
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):         #if any moves satisfy agent 1s first condition    
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1])) #sort by prey distance
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]                

        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator.pred_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps > 5000):
        fail = 2     
    return fail, path

In [9]:
def Agent3(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_3 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_3 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred = 0

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector[agent_pos-1] = 0
        for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not
    
    #print(belief_vector)
    
    while(fail == 0 and steps <= 1000):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob) #randomly selecting one of those positions and surveying it
        #print(possible_pred_pos)
        
        #survey and consequent belief update
        if possible_prey_pos == prey_3.pos: #if we survey node and find predator
            correct_pred+=1
            for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
                if i == prey_3.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
            prey_not = 1-max_prob #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[possible_prey_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/prey_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #print(belief_vector)
    #    print(sum(belief_vector))
    #    if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
    #        fail_list.append((1,steps))

        """quasi_1 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1""" #to ensure that belief vector sums up to 1, leaving no margin for error

        list_of_positions_max_chance = []
        for itr in range(len(belief_vector)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance) #randomly selecting one of those positions and assuming it as predator position

        #Once positions are decided, moving according to Agent 1
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predator_3.pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, predicted_prey_pos, g.graph)
            path_to_predator = BFS(neighbor, predator_3.pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey_3.pos): #if agent has already moved to location of prey
            break
        else: #if prey not caught, treat as survey and perform another belief update
            prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/prey_not
            
        #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #print(str(belief_vector) + ' first update during timestep')
        #print(belief_vector)
    #    print(sum(belief_vector))        
    #    if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
    #        fail_list.append((2,steps))        
        #print(sum(belief_vector))
        """quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error"""
            

        #move predator and prey
        prey_3.prey_move(g)

        if (prey_3.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator_3.pred_move(g, agent_pos)

        if(predator_3.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector)):
            mag_sum =  temp_belief_vector[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum += (temp_belief_vector[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector[i] = mag_sum
        #print(str(belief_vector) + ' last update before next timestep')
        #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))
        #print(belief_vector)
        #print(sum(belief_vector))        
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((3,steps))        
        """quasi_2 = sum(belief_vector)
        print(quasi_2)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_2"""
        #print(belief_vector)

    #if (len(fail_list) > 0):
    #    print("Fail List: " + str(fail_list))

    if(steps > 5000):
            fail = 2     
    return fail, path, correct_pred/(steps+1)

In [10]:
def Agent4(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_4 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_4 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred = 0

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector[agent_pos-1] = 0
        for i in range(len(belief_vector)):
            belief_vector[i] = belief_vector[i]/prey_not
    
    #print(belief_vector)
    
    while(fail == 0 and steps <= 1000):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob) #randomly selecting one of those positions and surveying it
        #print(possible_pred_pos)
        
        #survey and consequent belief update
        if possible_prey_pos == prey_4.pos: #if we survey node and find predator
            correct_pred += 1
            for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
                if i == prey_4.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
            prey_not = 1-max_prob #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[possible_prey_pos-1] = 0
            for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #print(belief_vector)
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((1,steps))

        """quasi_1 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1""" #to ensure that belief vector sums up to 1, leaving no margin for error

        list_of_positions_max_chance = []
        for itr in range(len(belief_vector)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance) #randomly selecting one of those positions and assuming it as predator position

        #after deciding position of prey, applying Agent 2's rules all the way

        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        all_possible_moves = []
        amazing_moves = []
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predator_4.pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]        

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, predicted_prey_pos, g.graph)
            path_to_pred = BFS(neighbour, predator_4.pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey_4.pos): #if agent has already moved to location of prey
            break
        else: #if prey not caught, treat as survey and perform another belief update
            prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not
            
        #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #print(str(belief_vector) + ' first update during timestep')
        #print(belief_vector)
        
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((2,steps))        
        #print(sum(belief_vector))
        """quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error"""
            

        #move predator and prey
        prey_4.prey_move(g)

        if (prey_4.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator_4.pred_move(g, agent_pos)

        if(predator_4.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector)):
            mag_sum = temp_belief_vector[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector[i] = mag_sum
        #print(str(belief_vector) + ' last update before next timestep')
        #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))
        #print(belief_vector)
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    fail_list.append((3,steps))        
        """quasi_2 = sum(belief_vector)
        print(quasi_2)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_2"""
        #print(belief_vector)

    #if (len(fail_list) > 0):
    #    print("Fail List: " + str(fail_list))

    if(steps > 5000):
            fail = 2     
    return fail, path, correct_pred/(steps+1)

In [11]:
def Agent5(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_5 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_5 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred = -1

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        if i == predator_5.pos-1:
            belief_vector[i] = 1
        else:
            belief_vector[i] = 0
    
    #print(sum(belief_vector))
    
    while(fail == 0 and steps <= 100):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)

        #chosing node with highest prob of having pred closest to agent position
        pat_len = 50
        best_positions = []
        for position in list_of_positions_max_prob:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_positions.append(position)
                else:
                    best_positions = []
                    best_positions.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))
        
        possible_pred_pos = random.choice(best_positions) #randomly selecting one of those positions and surveying it
        #print(possible_pred_pos)
        
        #surveying then updating belief
        if possible_pred_pos == predator_5.pos: #if we survey node and find predator
            correct_pred += 1
            for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
                if i == predator_5.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
                pred_not = 1-max_prob #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector[possible_pred_pos-1] = 0
                for i in range(len(belief_vector)):
                        belief_vector[i] = belief_vector[i]/pred_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #    fail_list.append((1,steps))


        quasi_1 = sum(belief_vector)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1 #to ensure that belief vector sums up to 1, leaving no margin for error

        list_of_positions_max_chance = []
        for itr in range(len(belief_vector)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)

        #chosing node with highest prob of having pred closest to us
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance) #randomly selecting one of those positions and assuming it as predator position
        #Agent 1 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, prey_5.pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, prey_5.pos, g.graph)
            path_to_predator = BFS(neighbor, predicted_pred_pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_5.pos):
            fail = 1 #if agent has already moved to location of pred
            break
        elif (agent_pos == prey_5.pos):
            break
        else: #treat as survey and update belief
            pred_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/pred_not
            
        #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #    fail_list.append((2,steps))        
        #print(str(belief_vector) + ' first update during timestep')
        #print(sum(belief_vector))

        quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error
            

        #move predator and prey
        prey_5.prey_move(g)

        if (prey_5.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator_5.pred_distracted_move(g, agent_pos)

        if(predator_5.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        #update belief for next timestep
        temp_belief_vector = belief_vector.copy()
        for i in range(len(belief_vector)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector[i] = mag_sum
        #print(str(belief_vector) + ' last update before next timestep')
        #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))

        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
            #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))
            #fail_list.append((3,steps+1))        

        quasi_2 = sum(belief_vector)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_2
        #print(belief_vector)
    
    #if (len(fail_list) > 0):
        #print("Fail List: " + str(fail_list))#
        
    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred/(steps)

In [12]:
def Agent6(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_6 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_6 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred = -1

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        if i == predator_6.pos-1:
            belief_vector[i] = 1
        else:
            belief_vector[i] = 0

    while(fail == 0 and steps <= 100):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)

        #chosing node with highest prob of having pred closest to agent
        pat_len = 50
        best_positions = []
        for position in list_of_positions_max_prob:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_positions.append(position)
                else:
                    best_positions = []
                    best_positions.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))        
        possible_pred_pos = random.choice(best_positions) #randomly selecting one of those positions and surveying it
        #print(possible_pred_pos)
        
        #survey and belief update
        if possible_pred_pos == predator_6.pos: #if we survey node and find predator
            correct_pred += 1
            for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
                if i == predator_6.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
                pred_not = 1-max_prob
                belief_vector[possible_pred_pos-1] = 0
                for i in range(len(belief_vector)):
                        belief_vector[i] = belief_vector[i]/pred_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(sum(belief_vector))

        quasi_1 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1
        #print(belief_vector)        

        list_of_positions_max_chance = []

        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)

        #chosing node with highest prob of having predator closest to us
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance) #randomly selecting one of those positions and surveying it
        #Agent 2 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        amazing_moves = []
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, prey_6.pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, prey_6.pos, g.graph)
            path_to_pred = BFS(neighbour, predicted_pred_pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]                

        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_6.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_6.pos):
            break
        else:#treat as survey and update belief
            pred_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/pred_not
            
            #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #print(str(belief_vector) + ' first update during timestep')
        #print(sum(belief_vector))
        quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error

        #move prey
        prey_6.prey_move(g)

        if (prey_6.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_6.pred_distracted_move(g, agent_pos)

        if(predator_6.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy()
        for i in range(len(belief_vector)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector[i] = mag_sum
        
        quasi = sum(belief_vector)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi
        #print(sum(belief_vector))
    
    if(steps > 5000):
            fail = 2     
    return fail, path, correct_pred/(steps)

In [28]:
def Agent7(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_7 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_7 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_7.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it   

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred == 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            if possible_prey_pos == prey_7.pos: #if we survey node and find predator
                correct_pred_prey += 1
                for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
                    if i == possible_prey_pos-1:
                        belief_vector_prey[i] = 1
                    else:
                        belief_vector_prey[i] = 0
            else:
                prey_not = 1-belief_vector_prey[possible_prey_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[possible_prey_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            if possible_pred_pos == predator_7.pos:
                correct_pred_predator += 1
            #if we find pred, survey same node for prey
                for i in range(len(belief_vector_pred)):
                    if i == possible_pred_pos-1:
                        belief_vector_pred[i] = 1
                    else:
                        belief_vector_pred[i] = 0

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            if possible_pred_pos == prey_7.pos:
                correct_pred_prey += 1
                for i in range(len(belief_vector_prey)):
                    if i == possible_pred_pos-1:
                        belief_vector_prey[i] = 1
                    else:
                        belief_vector_prey[i] = 0
            else:
                prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not

        quasi_1 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_1 #normalization for predator belief, to ensure sum doesnt exceed 1


        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)


        #Agent 1 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, predicted_prey_pos, g.graph)
            path_to_predator = BFS(neighbor, predicted_pred_pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_7.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_7.pos):
            break
        else:#treat as survey and perform belief update for both
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        #move prey
        prey_7.prey_move(g)

        if (prey_7.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_7.pred_distracted_move(g, agent_pos)

        if(predator_7.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief updates for both predator and prey for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [29]:
def Agent8(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_8 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_8 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_8.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        possible_pred_pos = random.choice(list_of_positions_max_prob_pred) #randomly selecting one of those positions and surveying it 
        

        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it  

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred == 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            if possible_prey_pos == prey_8.pos: #if we survey node and find predator
                correct_pred_prey += 1
                for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
                    if i == possible_prey_pos-1:
                        belief_vector_prey[i] = 1
                    else:
                        belief_vector_prey[i] = 0
            else:
                prey_not = 1-belief_vector_prey[possible_prey_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[possible_prey_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            if possible_pred_pos == predator_8.pos:
                correct_pred_predator += 1
            #if we find pred, survey same node for prey
                for i in range(len(belief_vector_pred)):
                    if i == possible_pred_pos-1:
                        belief_vector_pred[i] = 1
                    else:
                        belief_vector_pred[i] = 0

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            if possible_pred_pos == prey_8.pos:
                correct_pred_prey += 1
                for i in range(len(belief_vector_prey)):
                    if i == possible_pred_pos-1:
                        belief_vector_prey[i] = 1
                    else:
                        belief_vector_prey[i] = 0
            else:
                prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not

        quasi_1 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_1
        
        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        predicted_pred_pos = random.choice(list_of_positions_max_chance_pred)
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)

        #Agent 2 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        amazing_moves = []
        
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, predicted_prey_pos, g.graph)
            path_to_pred = BFS(neighbour, predicted_pred_pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_8.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_8.pos):
            break
        else: #treat as survey and perform belief update
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2

        #move prey
        prey_8.prey_move(g)

        if (prey_8.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_8.pred_distracted_move(g, agent_pos)

        if(predator_8.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for both prey and predator for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [15]:
graphs = 0

success_1 = []
max_win_step_1 = []
avg_win_step_1 = []
max_loss_step_1 = []
avg_loss_step_1 = []

success_2 = []
max_win_step_2 = []
avg_win_step_2 = []
max_loss_step_2 = []
avg_loss_step_2 = []

success_3 = []
max_win_step_3 = []
avg_win_step_3 = []
max_loss_step_3 = []
avg_loss_step_3 = []
prediction_3=[]

success_4 = []
max_win_step_4 = []
avg_win_step_4 = []
max_loss_step_4 = []
avg_loss_step_4 = []
prediction_4=[]

success_5 = []
max_win_step_5 = []
avg_win_step_5 = []
max_loss_step_5 = []
avg_loss_step_5 = []
prediction_5=[]

success_6 = []
max_win_step_6 = []
avg_win_step_6 = []
max_loss_step_6 = []
avg_loss_step_6 = []
prediction_6=[]

success_7 = []
max_win_step_7 = []
avg_win_step_7 = []
max_loss_step_7 = []
avg_loss_step_7 = []
prediction_prey_7=[]
prediction_predator_7=[]

success_8 = []
max_win_step_8 = []
avg_win_step_8 = []
max_loss_step_8 = []
avg_loss_step_8 = []
prediction_prey_8=[]
prediction_predator_8=[]


while graphs < 100:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()

    #print("Graph " + str(graphs) + " : " + str(g.graph))

    wins_1 = 0
    losses_1 = 0
    timeout_1 = 0
    win_dist_1 = []
    loss_dist_1 = []
    

    wins_2 = 0
    losses_2 = 0
    timeout_2 = 0
    win_dist_2 = []
    loss_dist_2 = []

    wins_3 = 0
    losses_3 = 0
    timeout_3 = 0
    win_dist_3 = []
    loss_dist_3 = []
    list_pred3 = []

    wins_4 = 0
    losses_4 = 0
    timeout_4 = 0
    win_dist_4 = []
    loss_dist_4 = []
    list_pred4 = []

    wins_5 = 0
    losses_5 = 0
    timeout_5 = 0
    win_dist_5 = []
    loss_dist_5 = []
    list_pred5 = []

    wins_6 = 0
    losses_6 = 0
    timeout_6 = 0
    win_dist_6 = []
    loss_dist_6 = []
    list_pred6=[]

    wins_7 = 0
    losses_7 = 0
    timeout_7 = 0
    win_dist_7 = []
    loss_dist_7 = []
    list_pred7prey = []
    list_pred7pred = []

    wins_8 = 0
    losses_8 = 0
    timeout_8 = 0
    win_dist_8 = []
    loss_dist_8 = []
    list_pred8prey = []
    list_pred8pred = []

    times=0

    while times < 30:

        agent_pos = random.randint(1,50)
        predator_pos = random.randint(1,50)
        while agent_pos == predator_pos:
            predator_pos = random.randint(1,50)
        prey_pos = random.randint(1,50)
        while agent_pos == prey_pos:
            prey_pos = random.randint(1,50)

        res_1, path_1 = Agent1(g, agent_pos, predator_pos, prey_pos)
        if(res_1 == 0):
            wins_1 += 1
            win_dist_1.append(len(path_1))
        elif(res_1 == 1):
            losses_1 += 1
            loss_dist_1.append(len(path_1))
        else:
            timeout_1 += 1

        res_2, path_2 = Agent2(g, agent_pos, predator_pos, prey_pos)
        if(res_2 == 0):
            wins_2 += 1
            win_dist_2.append(len(path_2))
        elif(res_2 == 1):
            losses_2 += 1
            loss_dist_2.append(len(path_2))
        else:
            timeout_2 += 1

        res_3, path_3, prediction3 = Agent3(g, agent_pos, predator_pos, prey_pos)
        list_pred3.append(prediction3)
        if(res_3 == 0):
            wins_3 += 1
            win_dist_3.append(len(path_3))
        elif(res_3 == 1):
            losses_3 += 1
            loss_dist_3.append(len(path_3))
        else:
            timeout_3 += 1

        res_4, path_4, prediction4 = Agent4(g, agent_pos, predator_pos, prey_pos)
        list_pred4.append(prediction4)
        if(res_4 == 0):
            wins_4 += 1
            win_dist_4.append(len(path_4))
        elif(res_4 == 1):
            losses_4 += 1
            loss_dist_4.append(len(path_4))
        else:
            timeout_4 += 1

        res_5, path_5, prediction5 = Agent5(g, agent_pos, predator_pos, prey_pos)
        list_pred5.append(prediction5)
        if(res_5 == 0):
            wins_5 += 1
            win_dist_5.append(len(path_5))
        elif(res_5 == 1):
            losses_5 += 1
            loss_dist_5.append(len(path_5))
        else:
            timeout_5 += 1

        res_6, path_6, prediction6 = Agent6(g, agent_pos, predator_pos, prey_pos)
        list_pred6.append(prediction6)
        if(res_6 == 0):
            wins_6 += 1
            win_dist_6.append(len(path_6))
        elif(res_6 == 1):
            losses_6 += 1
            loss_dist_6.append(len(path_6))
        else:
            timeout_6 += 1

        res_7, path_7, prediction_predator7, prediction_prey7 = Agent7(g, agent_pos, predator_pos, prey_pos)
        list_pred7pred.append(prediction_predator7)
        list_pred7prey.append(prediction_prey7)
        if(res_7 == 0):
            wins_7 += 1
            win_dist_7.append(len(path_7))
        elif(res_7 == 1):
            losses_7 += 1
            loss_dist_7.append(len(path_7))
        else:
            timeout_7 += 1

        res_8, path_8, prediction_predator8, prediction_prey8 = Agent8(g, agent_pos, predator_pos, prey_pos)
        list_pred8pred.append(prediction_predator8)
        list_pred8prey.append(prediction_prey8)        
        if(res_8 == 0):
            wins_8 += 1
            win_dist_8.append(len(path_8))
        elif(res_8 == 1):
            losses_8 += 1
            loss_dist_8.append(len(path_8))
        else:
            timeout_8 += 1
        
        times += 1

    #print('Number of wins: ' + str(wins))
    if (len(win_dist_1) > 0):
        max_win_step_1.append(max(win_dist_1)-1)
        avg_win_step_1.append((sum(win_dist_1)/wins_1)-1)        
    else:
        max_win_step_1.append("W")
        avg_win_step_1.append("W")
    if (len(loss_dist_1) > 0):
        max_loss_step_1.append(max(loss_dist_1)-1)
        avg_loss_step_1.append((sum(loss_dist_1)/losses_1)-1)        
    else:
        max_loss_step_1.append("L")
        avg_loss_step_1.append("L")   
    success_1.append((wins_1*100/times))


    if (len(win_dist_2) > 0):
        max_win_step_2.append(max(win_dist_2)-1)
        avg_win_step_2.append((sum(win_dist_2)/wins_2)-1)        
    else:
        max_win_step_2.append("W")
        avg_win_step_2.append("W")
    if (len(loss_dist_2) > 0):
        max_loss_step_2.append(max(loss_dist_2)-1)
        avg_loss_step_2.append((sum(loss_dist_2)/losses_2)-1)        
    else:
        max_loss_step_2.append("L")
        avg_loss_step_2.append("L")           
    success_2.append((wins_2*100/times))


    if (len(win_dist_3) > 0):
        max_win_step_3.append(max(win_dist_3)-1)
        avg_win_step_3.append((sum(win_dist_3)/wins_3)-1)        
    else:
        max_win_step_3.append("W")
        avg_win_step_3.append("W")
    if (len(loss_dist_3) > 0):
        max_loss_step_3.append(max(loss_dist_3)-1)
        avg_loss_step_3.append((sum(loss_dist_3)/losses_3)-1)        
    else:
        max_loss_step_3.append("L")
        avg_loss_step_3.append("L")
    prediction_3.append(sum(list_pred3)/len(list_pred3)*100)    
    success_3.append((wins_3*100/times))

    if (len(win_dist_4) > 0):
        max_win_step_4.append(max(win_dist_4)-1)
        avg_win_step_4.append((sum(win_dist_4)/wins_4)-1)        
    else:
        max_win_step_4.append("W")
        avg_win_step_4.append("W")
    if (len(loss_dist_4) > 0):
        max_loss_step_4.append(max(loss_dist_4)-1)
        avg_loss_step_4.append((sum(loss_dist_4)/losses_4)-1)        
    else:
        max_loss_step_4.append("L")
        avg_loss_step_4.append("L")         
    prediction_4.append(sum(list_pred4)/len(list_pred4)*100) 
    success_4.append((wins_4*100/times))

    if (len(win_dist_5) > 0):
        max_win_step_5.append(max(win_dist_5)-1)
        avg_win_step_5.append((sum(win_dist_5)/wins_5)-1)        
    else:
        max_win_step_5.append("W")
        avg_win_step_5.append("W")
    if (len(loss_dist_5) > 0):
        max_loss_step_5.append(max(loss_dist_5)-1)
        avg_loss_step_5.append((sum(loss_dist_5)/losses_5)-1)        
    else:
        max_loss_step_5.append("L")
        avg_loss_step_5.append("L")           
    prediction_5.append(sum(list_pred5)/len(list_pred5)*100) 
    success_5.append((wins_5*100/times))

    if (len(win_dist_6) > 0):
        max_win_step_6.append(max(win_dist_6)-1)
        avg_win_step_6.append((sum(win_dist_6)/wins_6)-1)        
    else:
        max_win_step_6.append("W")
        avg_win_step_6.append("W")
    if (len(loss_dist_6) > 0):
        max_loss_step_6.append(max(loss_dist_6)-1)
        avg_loss_step_6.append((sum(loss_dist_6)/losses_6)-1)        
    else:
        max_loss_step_6.append("L")
        avg_loss_step_6.append("L")      
    prediction_6.append(sum(list_pred6)/len(list_pred6)*100) 
    success_6.append((wins_6*100/times))

    if (len(win_dist_7) > 0):
        max_win_step_7.append(max(win_dist_7)-1)
        avg_win_step_7.append((sum(win_dist_7)/wins_7)-1)        
    else:
        max_win_step_7.append("W")
        avg_win_step_7.append("W")
    if (len(loss_dist_7) > 0):
        max_loss_step_7.append(max(loss_dist_7)-1)
        avg_loss_step_7.append((sum(loss_dist_7)/losses_7)-1)        
    else:
        max_loss_step_7.append("L")
        avg_loss_step_7.append("L")   
    prediction_prey_7.append(sum(list_pred7prey)/len(list_pred7prey)*100) 
    prediction_predator_7.append(sum(list_pred7pred)/len(list_pred7pred)*100) 
    success_7.append((wins_7*100/times))

    if (len(win_dist_8) > 0):
        max_win_step_8.append(max(win_dist_8)-1)
        avg_win_step_8.append((sum(win_dist_8)/wins_8)-1)        
    else:
        max_win_step_8.append("W")
        avg_win_step_8.append("W")
    if (len(loss_dist_8) > 0):
        max_loss_step_8.append(max(loss_dist_8)-1)
        avg_loss_step_8.append((sum(loss_dist_8)/losses_8)-1)        
    else:
        max_loss_step_8.append("L")
        avg_loss_step_8.append("L")       
    prediction_prey_8.append(sum(list_pred8prey)/len(list_pred8prey)*100) 
    prediction_predator_8.append(sum(list_pred8pred)/len(list_pred8pred)*100)
    success_8.append((wins_8*100/times))

    graphs += 1


print("Maximum Steps Taken to Win for Agent 1 for each Graph: " + str(max_win_step_1))
print("Average Steps taken to Win for Agent 1 for each Graph: " + str(avg_win_step_1))
#print("Maximum Steps Taken to Lose for Agent 1 for each Graph: " + str(max_loss_step_1))
#print("Average Steps Taken to Lose for Agent 1 for each Graph: " + str(avg_loss_step_1))
print("Success Rate for Agent 1 for each graph: " + str(success_1))
print()
print("Maximum Steps Taken to Win for Agent 2 for each Graph: " + str(max_win_step_2))
print("Average Steps taken to Win for Agent 2 for each Graph: " + str(avg_win_step_2))
#print("Maximum Steps Taken to Lose for Agent 2 for each Graph: " + str(max_loss_step_2))
#print("Average Steps Taken to Lose for Agent 2 for each Graph: " + str(avg_loss_step_2))
print("Success Rate for Agent 2 for each graph: " + str(success_2))
print()
print("Maximum Steps Taken to Win for Agent 3 for each Graph: " + str(max_win_step_3))
print("Average Steps taken to Win for Agent 3 for each Graph: " + str(avg_win_step_3))
#print("Maximum Steps Taken to Lose for Agent 3 for each Graph: " + str(max_loss_step_3))
#print("Average Steps Taken to Lose for Agent 3 for each Graph: " + str(avg_loss_step_3))
print("Success Rate for Agent 3 for each graph: " + str(success_3))
print("Correct Prediction of Preys Location in Agent 3: " + str(prediction_3))
print()
print("Maximum Steps Taken to Win for Agent 4 for each Graph: " + str(max_win_step_4))
print("Average Steps taken to Win for Agent 4 for each Graph: " + str(avg_win_step_4))
#print("Maximum Steps Taken to Lose for Agent 4 for each Graph: " + str(max_loss_step_4))
#print("Average Steps Taken to Lose for Agent 4 for each Graph: " + str(avg_loss_step_4))
print("Success Rate for Agent 4 for each graph: " + str(success_4))
print("Correct Prediction of Preys Location in Agent 4: " + str(prediction_4))
print()
print("Maximum Steps Taken to Win for Agent 5 for each Graph: " + str(max_win_step_5))
print("Average Steps taken to Win for Agent 5 for each Graph: " + str(avg_win_step_5))
#print("Maximum Steps Taken to Lose for Agent 5  for each Graph: " + str(max_loss_step_5))
#print("Average Steps Taken to Lose for Agent 5 for each Graph: " + str(avg_loss_step_5))
print("Success Rate for Agent 5 for each graph: " + str(success_5))
print("Correct Prediction of Predator's Location in Agent 5: " + str(prediction_5))
print()
print("Maximum Steps Taken to Win for Agent 6 for each Graph: " + str(max_win_step_6))
print("Average Steps taken to Win for Agent 6 for each Graph: " + str(avg_win_step_6))
#print("Maximum Steps Taken to Lose for Agent 6  for each Graph: " + str(max_loss_step_6))
#print("Average Steps Taken to Lose for Agent 6 for each Graph: " + str(avg_loss_step_6))
print("Success Rate for Agent 6 for each graph: " + str(success_6))
print("Correct Prediction of Predator's Location in Agent 6: " + str(prediction_6))
print()
print("Maximum Steps Taken to Win for Agent 7 for each Graph: " + str(max_win_step_7))
print("Average Steps taken to Win for Agent 7 for each Graph: " + str(avg_win_step_7))
#print("Maximum Steps Taken to Lose for Agent 7  for each Graph: " + str(max_loss_step_7))
#print("Average Steps Taken to Lose for Agent 7 for each Graph: " + str(avg_loss_step_7))
print("Success Rate for Agent 7 for each graph: " + str(success_7))
print("Correct Prediction of Preys Location in Agent 7: " + str(prediction_prey_7))
print("Correct Prediction of Predator's Location in Agent 7: " + str(prediction_predator_7))
print()
print("Maximum Steps Taken to Win for Agent 8 for each Graph: " + str(max_win_step_8))
print("Average Steps taken to Win for Agent 8 for each Graph: " + str(avg_win_step_8))
#print("Maximum Steps Taken to Lose for Agent 8  for each Graph: " + str(max_loss_step_8))
#print("Average Steps Taken to Lose for Agent 8 for each Graph: " + str(avg_loss_step_8))
print("Success Rate for Agent 8 for each graph: " + str(success_8))
print("Correct Prediction of Prey's Location in Agent 8: " + str(prediction_prey_8))
print("Correct Prediction of Predator's Location in Agent 8: " + str(prediction_predator_8))


Maximum Steps Taken to Win for Agent 1 for each Graph: [24, 21, 23, 27, 22, 29, 22, 35, 24, 30, 27, 31, 35, 22, 25, 28, 33, 25, 41, 28, 29, 32, 20, 31, 30, 28, 29, 27, 26, 19, 21, 27, 45, 30, 36, 24, 36, 30, 27, 30, 37, 32, 29, 29, 18, 24, 34, 60, 27, 46, 26, 27, 23, 28, 34, 29, 34, 39, 32, 23, 24, 23, 22, 26, 29, 20, 25, 28, 34, 28, 31, 53, 20, 39, 28, 33, 18, 25, 23, 27, 27, 18, 34, 24, 27, 33, 39, 32, 34, 27, 38, 27, 30, 28, 32, 28, 49, 24, 51, 27]
Average Steps taken to Win for Agent 1 for each Graph: [10.375, 9.708333333333334, 7.476190476190476, 13.566666666666666, 10.785714285714286, 12.076923076923077, 11.0, 12.62962962962963, 11.821428571428571, 12.142857142857142, 11.928571428571429, 13.178571428571429, 11.555555555555555, 9.74074074074074, 11.535714285714286, 11.592592592592593, 12.409090909090908, 12.464285714285714, 13.11111111111111, 14.392857142857142, 8.090909090909092, 11.620689655172415, 7.814814814814815, 13.076923076923077, 13.551724137931034, 13.6, 11.6333333333333

In [15]:
def Agent7_noisy(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_7 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_7 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_7.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it   

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred == 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            prey_survey_result = (possible_prey_pos == prey_7.pos)
            if prey_survey_result: #if it finds prey
                correct_pred_prey+=1
                chance = random.randint(1,100)
                if chance <= 90:  #90 percent chance give correct result
                    for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                        if i == possible_prey_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else: #10 percent chance report unoccupied
                    prey_not = 1-belief_vector_prey[possible_prey_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_prey_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not                                        
            else:
                prey_not = 1-belief_vector_prey[possible_prey_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[possible_prey_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            pred_survey_result = (possible_pred_pos == predator_7.pos)
            if pred_survey_result: #found predator
                correct_pred_predator+=1                
                chance = random.randint(1,100)
                if chance <= 90: #90 percent chance return occupied
            #if we find pred, survey same node for prey
                    for i in range(len(belief_vector_pred)):
                        if i == possible_pred_pos-1:
                            belief_vector_pred[i] = 1
                        else:
                            belief_vector_pred[i] = 0

                    if possible_pred_pos == prey_7.pos:
                        correct_pred_prey += 1
                        for i in range(len(belief_vector_prey)):
                            if i == possible_pred_pos-1:
                                correct_pred_prey+=1                                
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else:
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

                else:
                    pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                    belief_vector_pred[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_pred)):
                            belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not
                                    

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                prey_survey_result = (possible_pred_pos == prey_7.pos)
                if prey_survey_result: #if it finds prey
                    correct_pred_prey+=1                    
                    chance = random.randint(1,100)
                    if chance <= 90:  #90 percent chance give correct result
                        for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else: #10 percent chance report unoccupied
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not                                        
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not

        quasi_1 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_1 #normalization for predator belief, to ensure sum doesnt exceed 1


        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)


        #Agent 1 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, predicted_prey_pos, g.graph)
            path_to_predator = BFS(neighbor, predicted_pred_pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_7.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_7.pos):
            break
        else:#treat as survey and perform belief update for both
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        #move prey
        prey_7.prey_move(g)

        if (prey_7.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_7.pred_distracted_move(g, agent_pos)

        if(predator_7.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief updates for both predator and prey for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [16]:
def Agent8_noisy(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_8 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_8 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_8.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        possible_pred_pos = random.choice(list_of_positions_max_prob_pred) #randomly selecting one of those positions and surveying it 
        

        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it  

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred == 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            prey_survey_result = (possible_prey_pos == prey_8.pos)
            if prey_survey_result: #if it finds prey
                correct_pred_prey += 1
                chance = random.randint(1,100)
                if chance <= 90:  #90 percent chance give correct result
                    for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                        if i == possible_prey_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else: #10 percent chance report unoccupied
                    prey_not = 1-belief_vector_prey[possible_prey_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_prey_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not                                        
            else:
                prey_not = 1-belief_vector_prey[possible_prey_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[possible_prey_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            pred_survey_result = (possible_pred_pos == predator_8.pos)
            if pred_survey_result: #found predator
                correct_pred_predator += 1
                chance = random.randint(1,100)
                if chance <= 90: #90 percent chance return occupied
            #if we find pred, survey same node for prey
                    for i in range(len(belief_vector_pred)):
                        if i == possible_pred_pos-1:
                            belief_vector_pred[i] = 1
                        else:
                            belief_vector_pred[i] = 0

                    if possible_pred_pos == prey_8.pos:
                        correct_pred_prey += 1
                        for i in range(len(belief_vector_prey)):
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else:
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

                else:
                    pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                    belief_vector_pred[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_pred)):
                            belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not
                                    

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                prey_survey_result = (possible_pred_pos == prey_8.pos)
                if prey_survey_result: #if it finds prey
                    correct_pred_prey += 1
                    chance = random.randint(1,100)
                    if chance <= 90:  #90 percent chance give correct result
                        for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else: #10 percent chance report unoccupied
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not                                        
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not

        quasi_1 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_1
        
        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        predicted_pred_pos = random.choice(list_of_positions_max_chance_pred)
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)

        #Agent 2 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        amazing_moves = []
        
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, predicted_prey_pos, g.graph)
            path_to_pred = BFS(neighbour, predicted_pred_pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_8.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_8.pos):
            break
        else: #treat as survey and perform belief update
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2

        #move prey
        prey_8.prey_move(g)

        if (prey_8.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_8.pred_distracted_move(g, agent_pos)

        if(predator_8.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for both prey and predator for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [17]:
def Agent7_noisy_improved(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_7 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_7 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_7.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it   

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred == 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            prey_survey_result = (possible_prey_pos == prey_7.pos)
            if prey_survey_result: #if it finds prey
                correct_pred_prey += 1
                chance = random.randint(1,100)
                if chance <= 90:  #90 percent chance give correct result
                    for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                        if i == possible_prey_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else: #10 percent chance report unoccupied               
                #prey_not = 1-max_prob_prey #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_prey_pos-1])
                    belief_vector_prey[possible_prey_pos-1] = 0.1 * belief_vector_prey[possible_prey_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey                                      
            else:
                dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_prey_pos-1])
                belief_vector_prey[possible_prey_pos-1] = 0.1 * belief_vector_prey[possible_prey_pos-1]
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            pred_survey_result = (possible_pred_pos == predator_7.pos)
            if pred_survey_result: #found predator
                correct_pred_predator += 1
                chance = random.randint(1,100)
                if chance <= 90: #90 percent chance return occupied
            #if we find pred, survey same node for prey
                    for i in range(len(belief_vector_pred)):
                        if i == possible_pred_pos-1:
                            belief_vector_pred[i] = 1
                        else:
                            belief_vector_pred[i] = 0

                    if possible_pred_pos == prey_7.pos:
                        correct_pred_prey += 1
                        for i in range(len(belief_vector_prey)):
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else:
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

                else:
                    dividing_factor_pred = 1-(0.9*belief_vector_pred[possible_pred_pos-1])
                    belief_vector_pred[possible_pred_pos-1] = 0.1 * belief_vector_pred[possible_pred_pos-1]
                    for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/dividing_factor_pred
                        
                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                    belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
                                    

            else:
                #if we dont find pred, update belief vector for pred
                dividing_factor_pred = 1-(0.9*belief_vector_pred[possible_pred_pos-1])
                belief_vector_pred[possible_pred_pos-1] = 0.1 * belief_vector_pred[possible_pred_pos-1]
                for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/dividing_factor_pred

                prey_survey_result = (possible_pred_pos == prey_7.pos)
                if prey_survey_result: #if it finds prey
                    correct_pred_prey += 1
                    chance = random.randint(1,100)
                    if chance <= 90:  #90 percent chance give correct result
                        for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else: #10 percent chance report unoccupied
                        dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                        belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                        for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey                                       
                else:
                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                    belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
        
        
        """print(sum(belief_vector_pred))
        print(sum(belief_vector_prey))"""

        quasi_prey = sum(belief_vector_prey)
        for i in range(len(belief_vector_prey)):
            belief_vector_prey[i]/=quasi_prey #normalization for predator belief, to ensure sum doesnt exceed 1

        quasi_pred = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_pred #normalization for predator belief, to ensure sum doesnt exceed 1


        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)


        #Agent 1 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, predicted_prey_pos, g.graph)
            path_to_predator = BFS(neighbor, predicted_pred_pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_7.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_7.pos):
            break
        else:#treat as survey and perform belief update for both
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        #move prey
        prey_7.prey_move(g)

        if (prey_7.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_7.pred_distracted_move(g, agent_pos)

        if(predator_7.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief updates for both predator and prey for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [18]:
def Agent8_noisy_improved(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_8 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_8 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_8.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        possible_pred_pos = random.choice(list_of_positions_max_prob_pred) #randomly selecting one of those positions and surveying it 
        

        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it  

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred == 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            prey_survey_result = (possible_prey_pos == prey_8.pos)
            if prey_survey_result: #if it finds prey
                correct_pred_prey += 1
                chance = random.randint(1,100)
                if chance <= 90:  #90 percent chance give correct result
                    for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                        if i == possible_prey_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else: #10 percent chance report unoccupied                       
                #prey_not = 1-max_prob_prey #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_prey_pos-1])
                    belief_vector_prey[possible_prey_pos-1] = 0.1 * belief_vector_prey[possible_prey_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey                                      
            else:
                dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_prey_pos-1])
                belief_vector_prey[possible_prey_pos-1] = 0.1 * belief_vector_prey[possible_prey_pos-1]
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            pred_survey_result = (possible_pred_pos == predator_8.pos)
            if pred_survey_result: #found predator
                correct_pred_predator += 1
                chance = random.randint(1,100)
                if chance <= 90: #90 percent chance return occupied
            #if we find pred, survey same node for prey
                    for i in range(len(belief_vector_pred)):
                        if i == possible_pred_pos-1:
                            belief_vector_pred[i] = 1
                        else:
                            belief_vector_pred[i] = 0

                    if possible_pred_pos == prey_8.pos:
                        correct_pred_prey += 1
                        for i in range(len(belief_vector_prey)):
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else:
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

                else:
                    dividing_factor_pred = 1-(0.9*belief_vector_pred[possible_pred_pos-1])
                    belief_vector_pred[possible_pred_pos-1] = 0.1 * belief_vector_pred[possible_pred_pos-1]
                    for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/dividing_factor_pred

                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                    belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
                                    

            else:
                #if we dont find pred, update belief vector for pred
                dividing_factor_pred = 1-(0.9*belief_vector_pred[possible_pred_pos-1])
                belief_vector_pred[possible_pred_pos-1] = 0.1 * belief_vector_pred[possible_pred_pos-1]
                for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/dividing_factor_pred

                prey_survey_result = (possible_pred_pos == prey_8.pos)
                if prey_survey_result: #if it finds prey
                    correct_pred_prey += 1
                    chance = random.randint(1,100)
                    if chance <= 90:  #90 percent chance give correct result
                        for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else: #10 percent chance report unoccupied
                        dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                        belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                        for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey                                       
                else:
                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                    belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
        
        
        #print(sum(belief_vector_pred))
        #print(sum(belief_vector_prey))

        quasi_prey = sum(belief_vector_prey)
        for i in range(len(belief_vector_prey)):
            belief_vector_prey[i]/=quasi_prey #normalization for predator belief, to ensure sum doesnt exceed 1

        quasi_pred = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_pred #normalization for predator belief, to ensure sum doesnt exceed 1
        
        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        predicted_pred_pos = random.choice(list_of_positions_max_chance_pred)
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)

        #Agent 2 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        amazing_moves = []
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, predicted_prey_pos, g.graph)
            path_to_pred = BFS(neighbour, predicted_pred_pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_8.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_8.pos):
            break
        else: #treat as survey and perform belief update
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2

        #move prey
        prey_8.prey_move(g)

        if (prey_8.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_8.pred_distracted_move(g, agent_pos)

        if(predator_8.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for both prey and predator for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [20]:
def Agent9(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_9 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_9 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_9.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        possible_pred_pos = random.choice(list_of_positions_max_prob_pred) #randomly selecting one of those positions and surveying it 
        

        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it  

        max_prob_prey = max(belief_vector_prey)
        if (belief_vector_prey[possible_pred_pos-1] == max_prob_prey):
            possible_prey_pos = possible_pred_pos
        #print(max_prob)
        #getting max probability value in belief vector
        else:
            list_of_positions_max_prob_prey = []
            for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
                if(belief_vector_prey[itr] == max_prob_prey):
                    list_of_positions_max_prob_prey.append(itr+1)
            possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        if max_prob_pred >= 1: #if we are certain about predator position

            #predator belief update
            for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
                if i == possible_pred_pos-1:
                    belief_vector_pred[i] = 1
                else:
                    belief_vector_pred[i] = 0

            #survey node with highest probability of having prey and prey belief update when we are certain of pred loc
            prey_survey_result = (possible_prey_pos == prey_9.pos)
            if prey_survey_result: #if it finds prey
                correct_pred_prey += 1
                chance = random.randint(1,100)
                if chance <= 90:  #90 percent chance give correct result
                    for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                        if i == possible_prey_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else: #10 percent chance report unoccupied
                    if (possible_pred_pos == possible_prey_pos): #if prey position being surveyed is same as predator position (which we will be certain about due to probability being 1) and a nothing is seen in drone, assume prey is there
                        for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                            if i == possible_prey_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else:                        
                    #prey_not = 1-max_prob_prey #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_prey_pos-1])
                        belief_vector_prey[possible_prey_pos-1] = 0.1 * belief_vector_prey[possible_prey_pos-1]
                        for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey                                      
            else:
                dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_prey_pos-1])
                belief_vector_prey[possible_prey_pos-1] = 0.1 * belief_vector_prey[possible_prey_pos-1]
                for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
        
        
        #if we arent certain of pred position we survey node with highest probability of having predator for both prey and predator
        else:
            #if we arent certain of pred loc, survey node with highest probability
            pred_survey_result = (possible_pred_pos == predator_9.pos)
            if pred_survey_result: #found predator
                correct_pred_predator += 1
                chance = random.randint(1,100)
                if chance <= 90: #90 percent chance return occupied
            #if we find pred, survey same node for prey
                    for i in range(len(belief_vector_pred)):
                        if i == possible_pred_pos-1:
                            belief_vector_pred[i] = 1
                        else:
                            belief_vector_pred[i] = 0

                    if possible_pred_pos == prey_9.pos: 
                        correct_pred_prey += 1
                        for i in range(len(belief_vector_prey)):
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else:
                        prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                        belief_vector_prey[possible_pred_pos-1] = 0
                        for i in range(len(belief_vector_prey)):
                                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

                else:
                    dividing_factor_pred = 1-(0.9*belief_vector_pred[possible_pred_pos-1])
                    belief_vector_pred[possible_pred_pos-1] = 0.1 * belief_vector_pred[possible_pred_pos-1]
                    for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/dividing_factor_pred

                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                    belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
                                    

            else:
                #if we dont find pred, update belief vector for pred
                dividing_factor_pred = 1-(0.9*belief_vector_pred[possible_pred_pos-1])
                belief_vector_pred[possible_pred_pos-1] = 0.1 * belief_vector_pred[possible_pred_pos-1]
                for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/dividing_factor_pred

                prey_survey_result = (possible_pred_pos == prey_9.pos)
                if prey_survey_result: #if it finds prey
                    correct_pred_prey += 1
                    chance = random.randint(1,100)
                    if chance <= 90:  #90 percent chance give correct result
                        for i in range(len(belief_vector_prey)): #making probability of prey being in all other nodes 0
                            if i == possible_pred_pos-1:
                                belief_vector_prey[i] = 1
                            else:
                                belief_vector_prey[i] = 0
                    else: #10 percent chance report unoccupied
                        dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                        belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                        for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey                                       
                else:
                    dividing_factor_prey = 1-(0.9*belief_vector_prey[possible_pred_pos-1])
                    belief_vector_prey[possible_pred_pos-1] = 0.1 * belief_vector_prey[possible_pred_pos-1]
                    for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/dividing_factor_prey
        
        
        #print(sum(belief_vector_pred))
        #print(sum(belief_vector_prey))

        quasi_prey = sum(belief_vector_prey)
        for i in range(len(belief_vector_prey)):
            belief_vector_prey[i]/=quasi_prey #normalization for predator belief, to ensure sum doesnt exceed 1

        quasi_pred = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_pred #normalization for predator belief, to ensure sum doesnt exceed 1
        
        #belief updates done
        max_chance_pred = max(belief_vector_pred)
        list_of_positions_max_chance_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_chance_pred):
                list_of_positions_max_chance_pred.append(itr+1)


        #predicted_pred_pos = random.choice(list_of_positions_max_chance_pred)
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance)

        max_chance_prey = max(belief_vector_prey)
        list_of_positions_max_chance_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector_prey[itr] == max_chance_prey):
                list_of_positions_max_chance_prey.append(itr+1)
        #predicted_prey_pos = random.choice(list_of_positions_max_chance_prey)

        prey_option_distances = []

        for prey_positions in list_of_positions_max_chance_prey:
            prey_option_distances.append((len(BFS(agent_pos, prey_positions, g.graph)), prey_positions))

        sum_prey_option_distances = 0
        for spec in prey_option_distances:
            sum_prey_option_distances += spec[0]
        avg_dist_to_prey = sum_prey_option_distances/len(prey_option_distances)

        for prey_entities in prey_option_distances:
            if prey_entities[0]<avg_dist_to_prey:
                prey_option_distances.remove(prey_entities)

        predicted_prey = random.choice(prey_option_distances)
        predicted_prey_pos = predicted_prey[1]

        #Agent 2 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        amazing_moves = []
        path_to_prey = []
        path_to_pred = []
        prey_option_distances = []
        curr_dist_to_prey = len(BFS(agent_pos, predicted_prey_pos, g.graph))
        curr_dist_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))

        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, predicted_prey_pos, g.graph)
            path_to_pred = BFS(neighbour, predicted_pred_pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_9.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_9.pos):
            break
        else: #treat as survey and perform belief update
            pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_pred[agent_pos-1] = 0
            for i in range(len(belief_vector_pred)):
                    belief_vector_pred[i] = belief_vector_pred[i]/pred_not

            prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector_prey[agent_pos-1] = 0
            for i in range(len(belief_vector_prey)):
                    belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

        quasi_2 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_2

        #move prey
        prey_9.prey_move(g)

        if (prey_9.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_9.pred_distracted_move(g, agent_pos)

        if(predator_9.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for both prey and predator for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3


    if(steps > 5000):
        fail = 2     
    return fail, path, correct_pred_predator/(steps+1), correct_pred_prey/(steps+1)

In [21]:
graphs = 0

success_7 = []
max_win_step_7 = []
avg_win_step_7 = []
max_loss_step_7 = []
avg_loss_step_7 = []
prediction_prey_7=[]
prediction_predator_7=[]

success_8 = []
max_win_step_8 = []
avg_win_step_8 = []
max_loss_step_8 = []
avg_loss_step_8 = []
prediction_prey_8=[]
prediction_predator_8=[]

success_7_noisy = []
max_win_step_7_noisy = []
avg_win_step_7_noisy = []
max_loss_step_7_noisy = []
avg_loss_step_7_noisy = []
prediction_prey_7_noisy=[]
prediction_predator_7_noisy=[]

success_8_noisy = []
max_win_step_8_noisy = []
avg_win_step_8_noisy = []
max_loss_step_8_noisy = []
avg_loss_step_8_noisy = []
prediction_prey_8_noisy=[]
prediction_predator_8_noisy=[]

success_7_imp = []
max_win_step_7_imp = []
avg_win_step_7_imp = []
max_loss_step_7_imp = []
avg_loss_step_7_imp = []
prediction_prey_7_imp=[]
prediction_predator_7_imp=[]

success_8_imp = []
max_win_step_8_imp = []
avg_win_step_8_imp = []
max_loss_step_8_imp = []
avg_loss_step_8_imp = []
prediction_prey_8_imp=[]
prediction_predator_8_imp=[]

success_9 = []
max_win_step_9 = []
avg_win_step_9 = []
max_loss_step_9 = []
avg_loss_step_9 = []
prediction_prey_9=[]
prediction_predator_9=[]


while graphs < 100:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()

    #print("Graph " + str(graphs) + " : " + str(g.graph))

    wins_7 = 0
    losses_7 = 0
    timeout_7 = 0
    win_dist_7 = []
    loss_dist_7 = []
    list_pred7prey = []
    list_pred7pred = []

    wins_8 = 0
    losses_8 = 0
    timeout_8 = 0
    win_dist_8 = []
    loss_dist_8 = []
    list_pred8prey = []
    list_pred8pred = []

    wins_7_noisy = 0
    losses_7_noisy = 0
    timeout_7_noisy = 0
    win_dist_7_noisy = []
    loss_dist_7_noisy = []
    list_pred7prey_noisy = []
    list_pred7pred_noisy = []

    wins_8_noisy = 0
    losses_8_noisy = 0
    timeout_8_noisy = 0
    win_dist_8_noisy = []
    loss_dist_8_noisy = []
    list_pred8prey_noisy = []
    list_pred8pred_noisy = []

    wins_7_imp = 0
    losses_7_imp = 0
    timeout_7_imp = 0
    win_dist_7_imp = []
    loss_dist_7_imp = []
    list_pred7prey_imp = []
    list_pred7pred_imp = []

    wins_8_imp = 0
    losses_8_imp = 0
    timeout_8_imp = 0
    win_dist_8_imp = []
    loss_dist_8_imp = []
    list_pred8prey_imp = []
    list_pred8pred_imp = []

    wins_9 = 0
    losses_9 = 0
    timeout_9 = 0
    win_dist_9 = []
    loss_dist_9 = []
    list_pred9prey = []
    list_pred9pred = []

    times=0

    while times < 30:

        agent_pos = random.randint(1,50)
        predator_pos = random.randint(1,50)
        while agent_pos == predator_pos:
            predator_pos = random.randint(1,50)
        prey_pos = random.randint(1,50)
        while agent_pos == prey_pos:
            prey_pos = random.randint(1,50)

        res_7, path_7, prediction_predator7, prediction_prey7 = Agent7(g, agent_pos, predator_pos, prey_pos)
        list_pred7pred.append(prediction_predator7)
        list_pred7prey.append(prediction_prey7)
        if(res_7 == 0):
            wins_7 += 1
            win_dist_7.append(len(path_7))
        elif(res_7 == 1):
            losses_7 += 1
            loss_dist_7.append(len(path_7))
        else:
            timeout_7 += 1

        res_8, path_8, prediction_predator8, prediction_prey8 = Agent8(g, agent_pos, predator_pos, prey_pos)
        list_pred8pred.append(prediction_predator8)
        list_pred8prey.append(prediction_prey8)        
        if(res_8 == 0):
            wins_8 += 1
            win_dist_8.append(len(path_8))
        elif(res_8 == 1):
            losses_8 += 1
            loss_dist_8.append(len(path_8))
        else:
            timeout_8 += 1

        res_7noisy, path_7noisy, prediction_predator7noisy, prediction_prey7noisy = Agent7_noisy(g, agent_pos, predator_pos, prey_pos)
        list_pred7pred_noisy.append(prediction_predator7noisy)
        list_pred7prey_noisy.append(prediction_prey7noisy)
        if(res_7noisy == 0):
            wins_7_noisy += 1
            win_dist_7_noisy.append(len(path_7noisy))
        elif(res_7noisy == 1):
            losses_7_noisy += 1
            loss_dist_7_noisy.append(len(path_7noisy))
        else:
            timeout_7_noisy += 1

        res_8noisy, path_8noisy, prediction_predator8noisy, prediction_prey8noisy = Agent8_noisy(g, agent_pos, predator_pos, prey_pos)
        list_pred8pred_noisy.append(prediction_predator8noisy)
        list_pred8prey_noisy.append(prediction_prey8noisy)
        if(res_8noisy == 0):
            wins_8_noisy += 1
            win_dist_8_noisy.append(len(path_8noisy))
        elif(res_8noisy == 1):
            losses_8_noisy += 1
            loss_dist_8_noisy.append(len(path_8noisy))
        else:
            timeout_8_noisy += 1

        res_7_imp, path_7_imp, prediction_predator7_imp, prediction_prey7_imp = Agent7_noisy_improved(g, agent_pos, predator_pos, prey_pos)
        list_pred7pred_imp.append(prediction_predator7_imp)
        list_pred7prey_imp.append(prediction_prey7_imp)
        if(res_7_imp == 0):
            wins_7_imp += 1
            win_dist_7_imp.append(len(path_7_imp))
        elif(res_7_imp == 1):
            losses_7_imp += 1
            loss_dist_7_imp.append(len(path_7_imp))
        else:
            timeout_7_imp += 1

        res_8_imp, path_8_imp, prediction_predator8_imp, prediction_prey8_imp = Agent8_noisy_improved(g, agent_pos, predator_pos, prey_pos)
        list_pred8pred_imp.append(prediction_predator8_imp)
        list_pred8prey_imp.append(prediction_prey8_imp)        
        if(res_8_imp == 0):
            wins_8_imp += 1
            win_dist_8_imp.append(len(path_8_imp))
        elif(res_8_imp == 1):
            losses_8_imp += 1
            loss_dist_8_imp.append(len(path_8_imp))
        else:
            timeout_8_imp += 1

        res_9, path_9, prediction_predator9, prediction_prey9 = Agent9(g, agent_pos, predator_pos, prey_pos)
        list_pred9pred.append(prediction_predator9)
        list_pred9prey.append(prediction_prey9)
        if(res_9 == 0):
            wins_9 += 1
            win_dist_9.append(len(path_9))
        elif(res_9 == 1):
            losses_9 += 1
            loss_dist_9.append(len(path_9))
        else:
            timeout_9 += 1

        times += 1

    #print('Number of wins: ' + str(wins))
    if (len(win_dist_7) > 0):
        max_win_step_7.append(max(win_dist_7)-1)
        avg_win_step_7.append((sum(win_dist_7)/wins_7)-1)        
    else:
        max_win_step_7.append("W")
        avg_win_step_7.append("W")
    if (len(loss_dist_7) > 0):
        max_loss_step_7.append(max(loss_dist_7)-1)
        avg_loss_step_7.append((sum(loss_dist_7)/losses_7)-1)        
    else:
        max_loss_step_7.append("L")
        avg_loss_step_7.append("L")   
    prediction_prey_7.append(sum(list_pred7prey)/len(list_pred7prey)*100) 
    prediction_predator_7.append(sum(list_pred7pred)/len(list_pred7pred)*100) 
    success_7.append((wins_7*100/times))

    if (len(win_dist_8) > 0):
        max_win_step_8.append(max(win_dist_8)-1)
        avg_win_step_8.append((sum(win_dist_8)/wins_8)-1)        
    else:
        max_win_step_8.append("W")
        avg_win_step_8.append("W")
    if (len(loss_dist_8) > 0):
        max_loss_step_8.append(max(loss_dist_8)-1)
        avg_loss_step_8.append((sum(loss_dist_8)/losses_8)-1)        
    else:
        max_loss_step_8.append("L")
        avg_loss_step_8.append("L")       
    prediction_prey_8.append(sum(list_pred8prey)/len(list_pred8prey)*100) 
    prediction_predator_8.append(sum(list_pred8pred)/len(list_pred8pred)*100)
    success_8.append((wins_8*100/times))


    if (len(win_dist_7_noisy) > 0):
        max_win_step_7_noisy.append(max(win_dist_7_noisy)-1)
        avg_win_step_7_noisy.append((sum(win_dist_7_noisy)/wins_7_noisy)-1)        
    else:
        max_win_step_7_noisy.append("W")
        avg_win_step_7_noisy.append("W")
    if (len(loss_dist_7_noisy) > 0):
        max_loss_step_7_noisy.append(max(loss_dist_7_noisy)-1)
        avg_loss_step_7_noisy.append((sum(loss_dist_7_noisy)/losses_7_noisy)-1)        
    else:
        max_loss_step_7_noisy.append("L")
        avg_loss_step_7_noisy.append("L")
    prediction_prey_7_noisy.append(sum(list_pred7prey_noisy)/len(list_pred7prey_noisy)*100)
    prediction_predator_7_noisy.append(sum(list_pred7pred_noisy)/len(list_pred7pred_noisy)*100)   
    success_7_noisy.append((wins_7_noisy*100/times))

    if (len(win_dist_8_noisy) > 0):
        max_win_step_8_noisy.append(max(win_dist_8_noisy)-1)
        avg_win_step_8_noisy.append((sum(win_dist_8_noisy)/wins_8_noisy)-1)        
    else:
        max_win_step_8_noisy.append("W")
        avg_win_step_8_noisy.append("W")
    if (len(loss_dist_8_noisy) > 0):
        max_loss_step_8_noisy.append(max(loss_dist_8_noisy)-1)
        avg_loss_step_8_noisy.append((sum(loss_dist_8_noisy)/losses_8_noisy)-1)        
    else:
        max_loss_step_8_noisy.append("L")
        avg_loss_step_8_noisy.append("L")
    prediction_prey_8_noisy.append(sum(list_pred8prey_noisy)/len(list_pred8prey_noisy)*100)
    prediction_predator_8_noisy.append(sum(list_pred8pred_noisy)/len(list_pred8pred_noisy)*100)   
    success_8_noisy.append((wins_8_noisy*100/times))

    if (len(win_dist_7_imp) > 0):
        max_win_step_7_imp.append(max(win_dist_7_imp)-1)
        avg_win_step_7_imp.append((sum(win_dist_7_imp)/wins_7_imp)-1)        
    else:
        max_win_step_7_imp.append("W")
        avg_win_step_7_imp.append("W")
    prediction_prey_7_imp.append(sum(list_pred7prey_imp)/len(list_pred7pred_imp)*100)
    prediction_predator_7_imp.append(sum(list_pred7pred_imp)/len(list_pred7pred_imp)*100)     
    success_7_imp.append((wins_7_imp*100/times))

    if (len(win_dist_8_imp) > 0):
        max_win_step_8_imp.append(max(win_dist_8_imp)-1)
        avg_win_step_8_imp.append((sum(win_dist_8_imp)/wins_8_imp)-1)        
    else:
        max_win_step_8_imp.append("W")
        avg_win_step_8_imp.append("W")
    prediction_prey_8_imp.append(sum(list_pred8prey_imp)/len(list_pred8pred_imp)*100)
    prediction_predator_8_imp.append(sum(list_pred8pred_imp)/len(list_pred8pred_imp)*100)     
    success_8_imp.append((wins_8_imp*100/times))

    if (len(win_dist_9) > 0):
        max_win_step_9.append(max(win_dist_9)-1)
        avg_win_step_9.append((sum(win_dist_9)/wins_9)-1)        
    else:
        max_win_step_9.append("W")
        avg_win_step_9.append("W")
    if (len(loss_dist_9) > 0):
        max_loss_step_9.append(max(loss_dist_9)-1)
        avg_loss_step_9.append((sum(loss_dist_9)/losses_9)-1)        
    else:
        max_loss_step_9.append("L")
        avg_loss_step_9.append("L")   
    prediction_prey_9.append(sum(list_pred9prey)/len(list_pred9prey)*100) 
    prediction_predator_9.append(sum(list_pred9pred)/len(list_pred9pred)*100) 
    success_9.append((wins_9*100/times))

    graphs += 1


print("Maximum Steps Taken to Win for Agent 7 for each Graph: " + str(max_win_step_7))
print("Average Steps taken to Win for Agent 7 for each Graph: " + str(avg_win_step_7))
#print("Maximum Steps Taken to Lose for Agent 7  for each Graph: " + str(max_loss_step_7))
#print("Average Steps Taken to Lose for Agent 7 for each Graph: " + str(avg_loss_step_7))
print("Success Rate for Agent 7 for each graph: " + str(success_7))
print("Correct Prediction of Preys Location in Agent 7: " + str(prediction_prey_7))
print("Correct Prediction of Predator's Location in Agent 7: " + str(prediction_predator_7))
print()
print("Maximum Steps Taken to Win for Agent 8 for each Graph: " + str(max_win_step_8))
print("Average Steps taken to Win for Agent 8 for each Graph: " + str(avg_win_step_8))
#print("Maximum Steps Taken to Lose for Agent 8  for each Graph: " + str(max_loss_step_8))
#print("Average Steps Taken to Lose for Agent 8 for each Graph: " + str(avg_loss_step_8))
print("Success Rate for Agent 8 for each graph: " + str(success_8))
print("Correct Prediction of Prey's Location in Agent 8: " + str(prediction_prey_8))
print("Correct Prediction of Predator's Location in Agent 8: " + str(prediction_predator_8))
print()
print("Maximum Steps Taken to Win for Defective Drone Agent 7 for each Graph: " + str(max_win_step_7_noisy))
print("Average Steps taken to Win for Defective Drone Agent 7 for each Graph: " + str(avg_win_step_7_noisy))
#print("Maximum Steps Taken to Lose for Agent 3 for each Graph: " + str(max_loss_step_3))
#print("Average Steps Taken to Lose for Agent 3 for each Graph: " + str(avg_loss_step_3))
print("Success Rate for Defective Drone Agent 7 for each graph: " + str(success_7_noisy))
print("Correct Prediction of Preys Location in Defective Drone Agent 7: " + str(prediction_prey_7_noisy))
print("Correct Prediction of Predator's Location in Defective Drone Agent 7: " + str(prediction_predator_7_noisy))
print()
print("Maximum Steps Taken to Win for Defective Drone Agent 8 for each Graph: " + str(max_win_step_8_noisy))
print("Average Steps taken to Win for Defective Drone Agent 8 for each Graph: " + str(avg_win_step_8_noisy))
#print("Maximum Steps Taken to Lose for Agent 4 for each Graph: " + str(max_loss_step_4))
#print("Average Steps Taken to Lose for Agent 4 for each Graph: " + str(avg_loss_step_4))
print("Success Rate for Defective Drone Agent 8 for each graph: " + str(success_8_noisy))
print("Correct Prediction of Preys Location in Defective Drone Agent 8: " + str(prediction_prey_8_noisy))
print("Correct Prediction of Predator's Location in Defective Drone Agent 8: " + str(prediction_predator_8_noisy))
print()
print("Maximum Steps Taken to Win for Defective Drone Agent 7 with improved belief update for each Graph: " + str(max_win_step_7_imp))
print("Average Steps taken to Win for Defective Drone Agent 7 with improved belief update for each Graph: " + str(avg_win_step_7_imp))
#print("Maximum Steps Taken to Lose for Agent 4 for each Graph: " + str(max_loss_step_4))
#print("Average Steps Taken to Lose for Agent 4 for each Graph: " + str(avg_loss_step_4))
print("Success Rate for Defective Drone Agent 7 with improved belief update for each graph: " + str(success_7_imp))
print("Correct Prediction of Preys Location in Defective Drone Agent 7 with improved belief update: " + str(prediction_prey_7_imp))
print("Correct Prediction of Predator's Location in Defective Drone Agent 7 with improved belief update: " + str(prediction_predator_7_imp))
print()
print("Maximum Steps Taken to Win for Defective Drone Agent 8 with improved belief update for each Graph: " + str(max_win_step_8_imp))
print("Average Steps taken to Win for Defective Drone Agent 8 with improved belief update for each Graph: " + str(avg_win_step_8_imp))
#print("Maximum Steps Taken to Lose for Agent 4 for each Graph: " + str(max_loss_step_4))
#print("Average Steps Taken to Lose for Agent 4 for each Graph: " + str(avg_loss_step_4))
print("Success Rate for Defective Drone Agent 8 with improved belief update for each graph: " + str(success_8_imp))
print("Correct Prediction of Preys Location in Defective Drone Agent 8 with improved belief update: " + str(prediction_prey_8_imp))
print("Correct Prediction of Predator's Location in Defective Drone Agent 8 with improved belief update: " + str(prediction_predator_8_imp))
print()
print("Maximum Steps Taken to Win for Agent 9 for each Graph: " + str(max_win_step_9))
print("Average Steps taken to Win for Agent 9 for each Graph: " + str(avg_win_step_9))
#print("Maximum Steps Taken to Lose for Agent 8  for each Graph: " + str(max_loss_step_8))
#print("Average Steps Taken to Lose for Agent 8 for each Graph: " + str(avg_loss_step_8))
print("Success Rate for Agent 9 for each graph: " + str(success_9))
print("Correct Prediction of Prey's Location in Agent 9: " + str(prediction_prey_9))
print("Correct Prediction of Predator's Location in Agent 9: " + str(prediction_predator_9))

Maximum Steps Taken to Win for Agent 7 for each Graph: [72, 48, 103, 68, 90, 84, 62, 75, 56, 76, 62, 96, 60, 67, 78, 91, 73, 84, 77, 53, 84, 97, 96, 102, 66, 58, 95, 99, 56, 127, 118, 86, 142, 130, 48, 65, 54, 98, 50, 77, 107, 94, 82, 68, 68, 54, 52, 72, 137, 94, 84, 122, 88, 49, 86, 95, 76, 63, 69, 61, 137, 143, 53, 121, 115, 90, 106, 81, 87, 88, 126, 130, 112, 64, 66, 71, 126, 77, 72, 136, 53, 77, 93, 91, 111, 61, 116, 87, 81, 65, 102, 98, 84, 91, 69, 65, 111, 59, 114, 63]
Average Steps taken to Win for Agent 7 for each Graph: [26.956521739130434, 18.0, 28.77777777777778, 25.761904761904763, 26.51851851851852, 27.608695652173914, 30.347826086956523, 24.565217391304348, 30.15, 26.04, 23.16, 30.74074074074074, 22.782608695652176, 23.391304347826086, 17.35, 30.192307692307693, 21.40909090909091, 26.041666666666668, 21.291666666666668, 20.09090909090909, 28.842105263157894, 26.642857142857142, 18.708333333333332, 22.5, 24.08, 21.96, 28.03846153846154, 30.703703703703702, 20.4074074074074

In [38]:
def Agent7_bonus(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_7 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_7 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    surveys = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_7.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it   

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it

        curr_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))

        if(curr_to_pred <=3 or max_prob_pred >= 0.6):
            steps += 1
            #Agent 1 from here on
            neighbors = g.graph[agent_pos]
            #neighbors.append(agent_pos)
            # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
            # the neighbor they would move to for that path
            moves_1 = []
            moves_2 = []
            moves_3 = []
            moves_4 = []
            moves_5 = []
            moves_6 = []
            path_to_prey = []
            path_to_predator = []
            curr_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))
            curr_to_prey = len(BFS(agent_pos, possible_prey_pos, g.graph))

                
            for neighbor in neighbors:
                
                # shortest path from neighbor to prey and shortest path from neighbor to predator
                path_to_prey = BFS(neighbor, possible_prey_pos, g.graph)
                path_to_predator = BFS(neighbor, possible_pred_pos, g.graph)
                
                #print("Prey and Predator Moves")
                #print(path_to_prey)
                #print(path_to_predator)
                if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_1.append(tup)
                
                elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_2.append(tup)

                elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_3.append(tup)

                elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_4.append(tup)

                elif (len(path_to_predator) > curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_5.append(tup)

                elif (len(path_to_predator) == curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_6.append(tup)          

            
            #print()
            if(len(moves_1) > 0):
                moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_2) > 0):
                moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_3) > 0):
                moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_4) > 0):
                moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_5) > 0):
                moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_6) > 0):
                moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            if (agent_pos == predator_7.pos):
                fail = 1 #if agent has already moved to location of prey
                break
            elif (agent_pos == prey_7.pos):
                break
            else:#treat as survey and perform belief update for both
                pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_pred[agent_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[agent_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

            quasi_2 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        else:         
            surveys += 1   
            steps += 1
            #if we arent certain of pred loc, survey node with highest probability
            if possible_pred_pos == predator_7.pos:
                correct_pred_predator += 1
            #if we find pred, survey same node for prey
                for i in range(len(belief_vector_pred)):
                    if i == possible_pred_pos-1:
                        belief_vector_pred[i] = 1
                    else:
                        belief_vector_pred[i] = 0

                if possible_pred_pos == prey_7.pos:
                    correct_pred_prey += 1 
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                if possible_pred_pos == prey_7.pos:
                    correct_pred_prey += 1
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            quasi_1 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_1 #normalization for predator belief, to ensure sum doesnt exceed 1


        path.append(agent_pos)

        #move prey
        prey_7.prey_move(g)

        if (prey_7.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_7.pred_distracted_move(g, agent_pos)

        if(predator_7.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief updates for both predator and prey for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum


    if (steps > 5000):
        fail = 2
    if surveys > 0:     
        return fail, path, correct_pred_predator/surveys, correct_pred_prey/surveys
    else:
        return fail, path, 0, 0

In [39]:
def Agent8_bonus(g, a_pos, pred_pos, prey_pos):
    agent_pos = a_pos #initializing agent
    predator_8 = Predator(pred_pos) #initializing predator in location where agent doesnt exist
    prey_8 = Prey(prey_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    correct_pred_prey = 0
    correct_pred_predator = 0
    surveys = 0
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_8.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        possible_pred_pos = random.choice(list_of_positions_max_prob_pred) #randomly selecting one of those positions and surveying it 
        

        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it  

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        curr_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))

        if(curr_to_pred <= 3 or max_prob_pred >= 0.6):
            steps += 1
            neighbors = g.graph[agent_pos]
            #neighbors.append(agent_pos)
            # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
            # the neighbor they would move to for that path
            amazing_moves = []
            
            path_to_prey = []
            path_to_pred = []
            curr_dist_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))
            curr_dist_to_prey = len(BFS(agent_pos, possible_prey_pos, g.graph))

            all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

            
            for neighbour in neighbors:
                path_to_prey = BFS(neighbour, possible_prey_pos, g.graph)
                path_to_pred = BFS(neighbour, possible_pred_pos, g.graph)

                #keeping all moves handy
                tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
                all_possible_moves.append(tup2)
                
                #making list of moves which only satisfy 1st condition of Agent 1
                if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                    tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                    amazing_moves.append(tup3)                


            if(len(amazing_moves) > 0):             
                moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]


            else: #dip into bag of all possible moves

                if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                    moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                    move_choice=[]
                    for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                        if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                            move_choice.append(move)
                    neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                    agent_pos = neighbour_choice[2]

                else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                    moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                    move_choice=[]
                    for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                        if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                            move_choice.append(move)
                    neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                    agent_pos = neighbour_choice[2]

            if (agent_pos == predator_8.pos):
                fail = 1 #if agent has already moved to location of prey
                break
            elif (agent_pos == prey_8.pos):
                break
            else:#treat as survey and perform belief update for both
                pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_pred[agent_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[agent_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

            quasi_2 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        else:            
            surveys += 1
            steps += 1
            #if we arent certain of pred loc, survey node with highest probability
            if possible_pred_pos == predator_8.pos:
                correct_pred_predator += 1
            #if we find pred, survey same node for prey
                for i in range(len(belief_vector_pred)):
                    if i == possible_pred_pos-1:
                        belief_vector_pred[i] = 1
                    else:
                        belief_vector_pred[i] = 0

                if possible_pred_pos == prey_8.pos:
                    correct_pred_prey += 1
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                if possible_pred_pos == prey_8.pos:
                    correct_pred_prey += 1 
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            quasi_1 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_1 #normalization for predator belief, to ensure sum doesnt exceed 1


        path.append(agent_pos)
        
        
        #move prey
        prey_8.prey_move(g)

        if (prey_8.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_8.pred_distracted_move(g, agent_pos)

        if(predator_8.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for both prey and predator for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0
            i_neighbours = g.graph[i+1]
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke]
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours)))
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes))
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))
            belief_vector_prey[i] = mag_sum

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3


    if (steps > 5000):
        fail = 2
    if surveys > 0:     
        return fail, path, correct_pred_predator/surveys, correct_pred_prey/surveys
    else:
        return fail, path, 0, 0

In [40]:
graphs = 0

success_7 = []
max_win_step_7 = []
avg_win_step_7 = []
max_loss_step_7 = []
avg_loss_step_7 = []
prediction_prey_7=[]
prediction_predator_7=[]

success_8 = []
max_win_step_8 = []
avg_win_step_8 = []
max_loss_step_8 = []
avg_loss_step_8 = []
prediction_prey_8=[]
prediction_predator_8=[]

success_7_bonus = []
max_win_step_7_bonus = []
avg_win_step_7_bonus = []
max_loss_step_7_bonus = []
avg_loss_step_7_bonus = []
prediction_prey_7_bonus=[]
prediction_predator_7_bonus=[]

success_8_bonus = []
max_win_step_8_bonus = []
avg_win_step_8_bonus = []
max_loss_step_8_bonus = []
avg_loss_step_8_bonus = []
prediction_prey_8_bonus=[]
prediction_predator_8_bonus=[]

while graphs < 100:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()

    #print("Graph " + str(graphs) + " : " + str(g.graph))

    wins_7 = 0
    losses_7 = 0
    timeout_7 = 0
    win_dist_7 = []
    loss_dist_7 = []
    list_pred7prey = []
    list_pred7pred = []

    wins_8 = 0
    losses_8 = 0
    timeout_8 = 0
    win_dist_8 = []
    loss_dist_8 = []
    list_pred8prey = []
    list_pred8pred = []

    wins_7_bonus = 0
    losses_7_bonus = 0
    timeout_7_bonus = 0
    win_dist_7_bonus = []
    loss_dist_7_bonus = []
    list_pred7prey_bonus = []
    list_pred7pred_bonus = []

    wins_8_bonus = 0
    losses_8_bonus = 0
    timeout_8_bonus = 0
    win_dist_8_bonus = []
    loss_dist_8_bonus = []
    list_pred8prey_bonus = []
    list_pred8pred_bonus = []

    times=0

    while times < 30:

        agent_pos = random.randint(1,50)
        predator_pos = random.randint(1,50)
        while agent_pos == predator_pos:
            predator_pos = random.randint(1,50)
        prey_pos = random.randint(1,50)
        while agent_pos == prey_pos:
            prey_pos = random.randint(1,50)

        res_7, path_7, prediction_predator7, prediction_prey7 = Agent7(g, agent_pos, predator_pos, prey_pos)
        list_pred7pred.append(prediction_predator7)
        list_pred7prey.append(prediction_prey7)
        if(res_7 == 0):
            wins_7 += 1
            win_dist_7.append(len(path_7))
        elif(res_7 == 1):
            losses_7 += 1
            loss_dist_7.append(len(path_7))
        else:
            timeout_7 += 1

        res_8, path_8, prediction_predator8, prediction_prey8 = Agent8(g, agent_pos, predator_pos, prey_pos)
        list_pred8pred.append(prediction_predator8)
        list_pred8prey.append(prediction_prey8)        
        if(res_8 == 0):
            wins_8 += 1
            win_dist_8.append(len(path_8))
        elif(res_8 == 1):
            losses_8 += 1
            loss_dist_8.append(len(path_8))
        else:
            timeout_8 += 1

        res_7bonus, path_7bonus, prediction_predator7bonus, prediction_prey7bonus= Agent7_bonus(g, agent_pos, predator_pos, prey_pos)
        list_pred7pred_bonus.append(prediction_predator7bonus)
        list_pred7prey_bonus.append(prediction_prey7bonus)
        if(res_7bonus == 0):
            wins_7_bonus += 1
            win_dist_7_bonus.append(len(path_7bonus))
        elif(res_7bonus == 1):
            losses_7_bonus += 1
            loss_dist_7_bonus.append(len(path_7bonus))
        else:
            timeout_7_bonus += 1

        res_8bonus, path_8bonus, prediction_predator8bonus, prediction_prey8bonus = Agent8_bonus(g, agent_pos, predator_pos, prey_pos)
        list_pred8pred_bonus.append(prediction_predator8bonus)
        list_pred8prey_bonus.append(prediction_prey8bonus)
        if(res_8bonus == 0):
            wins_8_bonus += 1
            win_dist_8_bonus.append(len(path_8bonus))
        elif(res_8bonus == 1):
            losses_8_bonus += 1
            loss_dist_8_bonus.append(len(path_8bonus))
        else:
            timeout_8_bonus += 1

        times += 1

    #print('Number of wins: ' + str(wins))
    if (len(win_dist_7) > 0):
        max_win_step_7.append(max(win_dist_7)-1)
        avg_win_step_7.append((sum(win_dist_7)/wins_7)-1)        
    else:
        max_win_step_7.append("W")
        avg_win_step_7.append("W")
    if (len(loss_dist_7) > 0):
        max_loss_step_7.append(max(loss_dist_7)-1)
        avg_loss_step_7.append((sum(loss_dist_7)/losses_7)-1)        
    else:
        max_loss_step_7.append("L")
        avg_loss_step_7.append("L")   
    prediction_prey_7.append(sum(list_pred7prey)/len(list_pred7prey)*100) 
    prediction_predator_7.append(sum(list_pred7pred)/len(list_pred7pred)*100) 
    success_7.append((wins_7*100/times))

    if (len(win_dist_8) > 0):
        max_win_step_8.append(max(win_dist_8)-1)
        avg_win_step_8.append((sum(win_dist_8)/wins_8)-1)        
    else:
        max_win_step_8.append("W")
        avg_win_step_8.append("W")
    if (len(loss_dist_8) > 0):
        max_loss_step_8.append(max(loss_dist_8)-1)
        avg_loss_step_8.append((sum(loss_dist_8)/losses_8)-1)        
    else:
        max_loss_step_8.append("L")
        avg_loss_step_8.append("L")       
    prediction_prey_8.append(sum(list_pred8prey)/len(list_pred8prey)*100) 
    prediction_predator_8.append(sum(list_pred8pred)/len(list_pred8pred)*100)
    success_8.append((wins_8*100/times))


    if (len(win_dist_7_bonus) > 0):
        max_win_step_7_bonus.append(max(win_dist_7_bonus)-1)
        avg_win_step_7_bonus.append((sum(win_dist_7_bonus)/wins_7_bonus)-1)        
    else:
        max_win_step_7_bonus.append("W")
        avg_win_step_7_bonus.append("W")
    if (len(loss_dist_7_bonus) > 0):
        max_loss_step_7_bonus.append(max(loss_dist_7_bonus)-1)
        avg_loss_step_7_bonus.append((sum(loss_dist_7_bonus)/losses_7_bonus)-1)        
    else:
        max_loss_step_7_bonus.append("L")
        avg_loss_step_7_bonus.append("L")
    prediction_prey_7_bonus.append(sum(list_pred7prey_bonus)/len(list_pred7prey_bonus)*100)
    prediction_predator_7_bonus.append(sum(list_pred7pred_bonus)/len(list_pred7pred_bonus)*100)   
    success_7_bonus.append((wins_7_bonus*100/times))

    if (len(win_dist_8_bonus) > 0):
        max_win_step_8_bonus.append(max(win_dist_8_bonus)-1)
        avg_win_step_8_bonus.append((sum(win_dist_8_bonus)/wins_8_bonus)-1)        
    else:
        max_win_step_8_bonus.append("W")
        avg_win_step_8_bonus.append("W")
    if (len(loss_dist_8_bonus) > 0):
        max_loss_step_8_bonus.append(max(loss_dist_8_bonus)-1)
        avg_loss_step_8_bonus.append((sum(loss_dist_8_bonus)/losses_8_bonus)-1)        
    else:
        max_loss_step_8_bonus.append("L")
        avg_loss_step_8_bonus.append("L")
    prediction_prey_8_bonus.append(sum(list_pred8prey_bonus)/len(list_pred8prey_bonus)*100)
    prediction_predator_8_bonus.append(sum(list_pred8pred_bonus)/len(list_pred8pred_bonus)*100)   
    success_8_bonus.append((wins_8_bonus*100/times))

    graphs += 1


print("Maximum Steps Taken to Win for Agent 7 for each Graph: " + str(max_win_step_7))
print("Average Steps taken to Win for Agent 7 for each Graph: " + str(avg_win_step_7))
#print("Maximum Steps Taken to Lose for Agent 7  for each Graph: " + str(max_loss_step_7))
#print("Average Steps Taken to Lose for Agent 7 for each Graph: " + str(avg_loss_step_7))
print("Success Rate for Agent 7 for each graph: " + str(success_7))
print("Correct Prediction of Preys Location in Agent 7: " + str(prediction_prey_7))
print("Correct Prediction of Predator's Location in Agent 7: " + str(prediction_predator_7))
print()
print("Maximum Steps Taken to Win for Agent 8 for each Graph: " + str(max_win_step_8))
print("Average Steps taken to Win for Agent 8 for each Graph: " + str(avg_win_step_8))
#print("Maximum Steps Taken to Lose for Agent 8  for each Graph: " + str(max_loss_step_8))
#print("Average Steps Taken to Lose for Agent 8 for each Graph: " + str(avg_loss_step_8))
print("Success Rate for Agent 8 for each graph: " + str(success_8))
print("Correct Prediction of Prey's Location in Agent 8: " + str(prediction_prey_8))
print("Correct Prediction of Predator's Location in Agent 8: " + str(prediction_predator_8))
print()
print("Maximum Steps Taken to Win for Agent 7 Bonus for each Graph: " + str(max_win_step_7_bonus))
print("Average Steps taken to Win for Agent 7 Bonus for each Graph: " + str(avg_win_step_7_bonus))
#print("Maximum Steps Taken to Lose for Agent 3 for each Graph: " + str(max_loss_step_3))
#print("Average Steps Taken to Lose for Agent 3 for each Graph: " + str(avg_loss_step_3))
print("Success Rate for Agent 7 Bonus for each graph: " + str(success_7_bonus))
print("Correct Prediction of Preys Location in Agent 7 Bonus: " + str(prediction_prey_7_bonus))
print("Correct Prediction of Predator's Location in Agent 7 Bonus: " + str(prediction_predator_7_bonus))
print()
print("Maximum Steps Taken to Win for Agent 8 Bonus for each Graph: " + str(max_win_step_8_bonus))
print("Average Steps taken to Win for Agent 8 Bonus for each Graph: " + str(avg_win_step_8_bonus))
#print("Maximum Steps Taken to Lose for Agent 4 for each Graph: " + str(max_loss_step_4))
#print("Average Steps Taken to Lose for Agent 4 for each Graph: " + str(avg_loss_step_4))
print("Success Rate for Agent 8 Bonus for each graph: " + str(success_8_bonus))
print("Correct Prediction of Preys Location in Agent 8 Bonus: " + str(prediction_prey_8_bonus))
print("Correct Prediction of Predator's Location in Agent 8 Bonus: " + str(prediction_predator_8_bonus))

Maximum Steps Taken to Win for Agent 7 for each Graph: [63, 92, 78, 74, 49, 81, 129, 73, 72, 123, 49, 70, 80, 86, 99, 61, 98, 67, 120, 106, 97, 145, 124, 88, 81, 73, 92, 69, 65, 102, 106, 70, 145, 78, 94, 104, 89, 68, 101, 89, 74, 66, 60, 85, 66, 104, 78, 98, 111, 94, 98, 91, 53, 102, 98, 115, 58, 87, 116, 72, 67, 46, 65, 73, 88, 129, 126, 61, 101, 113, 79, 94, 77, 147, 69, 70, 116, 88, 96, 43, 54, 132, 95, 78, 77, 110, 111, 133, 40, 93, 105, 71, 64, 92, 116, 94, 95, 80, 68, 66]
Average Steps taken to Win for Agent 7 for each Graph: [25.375, 25.12, 34.125, 24.818181818181817, 20.96, 21.238095238095237, 29.954545454545453, 24.666666666666668, 25.307692307692307, 34.391304347826086, 19.666666666666668, 16.7, 33.19230769230769, 30.59090909090909, 37.291666666666664, 22.666666666666668, 26.545454545454547, 24.6, 30.08, 23.130434782608695, 27.227272727272727, 35.77272727272727, 31.049999999999997, 29.5, 29.285714285714285, 21.47826086956522, 29.545454545454547, 24.35, 19.73913043478261, 33.